# 1. Motivation


### What is your dataset?


My data set is based on the dota 2 pro players the dotamax.com offers at http://dotamax.com/player/pro/. The initial set had 750 nodes. 

### Why did you choose this/these particular dataset(s)?
I like the game a lot and was curious about what facts i can discover by rearching the data.

### What was your goal for the end user's experience?
I wanted the user to find out the following:
    * Player the played with most other pros
    * Best possible team based on statistics
    * Best player based on the heroes scores 
    * Best player based on their ranking and how similar he is with the other players he played with or against.

# 2. Basic stats. Let's understand the dataset better


### Write about your choices in data cleaning and preprocessing
Now based on every player, I used the https://docs.opendota.com/ API to retrieve the edges for every node. Everytime a player played with anoter pro, I created an edge. 
I removed the nodes that had no edges.


In [9]:
r1 = open('pros.txt','r')
s =""
for line in r1:
    s= s + line
k = s.split('</div><div><span style="color:#555;">')
print "Number of pros inside the HTML: ", len(k)-1
prosDict={}
for i in range(0,len(k)):
    try:
        l = k[i]
        z = l.split('href="/player/detail/')
        idPlayer = z[1].split('"><img class="match-avatars-img" style="')[0]
        y =z[1].split('</img><div style="margin-top: 8px;">') 
        y1 = y[len(y)-1]
        namePlayer = y1.split('<span class="glyphicon glyphicon-ok"')[0]
        namePlayer.strip()
        if '</font>' in namePlayer:
            namePlayer = namePlayer.split('</font>')[1]

        prosDict[idPlayer] = namePlayer.strip()
    except:
        m=0
print "Number of pros inside the dictionary", len(prosDict)
print "printing some examples of players and their id"
counter = 0
for i in prosDict:
    print i, prosDict[i]
    counter=counter+1
    if counter==10:
        break

Number of pros inside the HTML:  765
Number of pros inside the dictionary 765
printing some examples of players and their id
188384233 JJ-
76904792 poloson
43720230 Aghanim
112127585 판다맨
169141480 Varizh
154715080 Abed
86802844 Mag
139174891 897
31763321 Jellopy
107462128 MasterRamen


# 3. Tools, theory and analysis. Describe the process of theory to insight


### Adding the edges

In [10]:
import networkx as nx
G=nx.Graph()
import requests
maximum = 2000
count=0
for i in prosDict:
    count=count+1
    baseurl = "https://api.opendota.com/api/players/"+  str(i)+"/pros"
    response = requests.get(baseurl)
    data = response.json()
    for j in range(1,len(data)):

        fellowProId = str(data[j]['account_id'])
        games= data[j]['games']
        with_win = data[j]['with_win']
        with_games = data[j]['with_games']
        against_win = data[j]['against_win']
        against_games = data[j]['against_games']
        if G.has_edge(i, fellowProId):
            if G[i][fellowProId]['games']>games:
                G.remove_edge(i,fellowProId)
                w = 2000 - games
                if w<=0:
                    w= 1
                G.add_edge(i, fellowProId, weight = w)
                G[i][fellowProId]['games'] = games
                G[i][fellowProId]['with_win'] = with_win
                G[i][fellowProId]['with_games'] = with_games
                G[i][fellowProId]['against_win'] = against_win
                G[i][fellowProId]['against_games']=against_games
        else: 
            if fellowProId in prosDict:
                w = 2000 - games
                if w<=0:
                    w= 1
                G.add_edge(i, fellowProId, weight = w)
                G[i][fellowProId]['games'] = games
                G[i][fellowProId]['with_win'] = with_win
                G[i][fellowProId]['with_games'] = with_games
                G[i][fellowProId]['against_win'] = against_win
                G[i][fellowProId]['against_games']=against_games

In [11]:
print "Number of edges before deleting some: ",len(G.edges())

Number of edges before deleting some:  87234


### After having all the edges and nice data stats like their winrate together, I removed all the edges between nodes that played less then 50 games together. 

In [16]:
H= G.copy()
for edge1,edge2 in H.edges():
    if H[edge1][edge2]!='name':
        if H[edge1][edge2]['games']<50:
            H.remove_edge(edge1,edge2)
print "NUmber of edges with more then 50 games:", len(H.edges())

NUmber of edges with more then 50 games: 7905


### Getting regions for each player  and save it in a dictionary:regionsDict with the player's id as key


In [ ]:
count =0
regionsDict={}
for i in prosDict:
    
    count=count+1
    print (count/765.0) 
    baseurl = "https://api.opendota.com/api/players/"+  str(i)+"/counts"
    response = requests.get(baseurl)
    data = response.json()

    maxi= 0
    region = 0
    try:
        for k in data["region"]:
            if data["region"][k]["games"]>maxi:
                maxi = data["region"][k]["games"]
                region = int(k)
        regionsDict[i] = region
    except:
        regionsDict[i] = 25
        print i, "error"

### Getting their ranking   and save it in a dictionary:mmrDict with the player's id as key

In [ ]:
mmrDict = {}
for i in prosDict:
    count=count+1
    baseurl = "https://api.opendota.com/api/players/"+  str(i)
    response = requests.get(baseurl)
    data = response.json()
    try:
        soloMMr = data["solo_competitive_rank"]
        if soloMMr==None:
            soloMMr = data["mmr_estimate"]["estimate"]
        mmrDict[i]= int(soloMMr)
    except:

        print "error"
        mmrDict[i]=-1

### Getting ranking of players for each hero  

* First I needed to get all heroes, i saved those in heroesDict
* now base on the heroes and players, i created a network called heroesGraph. 

In [ ]:
##https://api.opendota.com/api/heroes
heroesDict = {}
baseurl = "https://api.opendota.com/api/heroes/"
response = requests.get(baseurl)
data = response.json()
for i in data:
    heroesDict[i['id']]= i['localized_name']
heroesGraph = nx.Graph()
for i in heroesDict:
    baseurl = "https://api.opendota.com/api/rankings?hero_id=" + str(i)
    response = requests.get(baseurl)
    data = response.json()
    for k in data['rankings']:
        accountId = k['account_id']
        if accountId in prosDict:
            heroesGraph.add_edge(i, accountId)

### Calculating the best teams


### This is the algorithm is used for calculating the best teams

In [14]:
L = G.copy()
for edge1,edge2 in L.edges():
    if L[edge1][edge2]["with_games"]<50:
        L.remove_edge(edge1,edge2)
for edge1,edge2 in L.edges():
    if (L[edge1][edge2]["with_win"]+0.0)/L[edge1][edge2]["with_games"]<0.65:
        L.remove_edge(edge1,edge2)

dictWinners ={}
counter = 0
for edge1,edge2 in L.edges():
    #print edge1,edge2,L[edge1][edge2]["with_win"],L[edge1][edge2]["with_games"],(L[edge1][edge2]["with_win"]+0.0)/L[edge1][edge2]["with_games"]
    s1 = set(L.neighbors(edge1))
    s1.add(edge1)

    s2 = set(L.neighbors(edge2))
    s2.add(edge2)
    s1=s1.intersection(s2)
    for i in s1:
        s3 = set(L.neighbors(i))
        s3 =s3.intersection(s1)
        for k in s3:
            if edge1 in L.neighbors(k) and edge2 in L.neighbors(k):
                s4 = set(L.neighbors(k))
                s4 = s4.intersection(s3)
                for j in s4:
                    if edge1 in L.neighbors(j) and edge2 in L.neighbors(j) and i in L.neighbors(j):
                        list1 = [edge1,edge2,i,k,j]
                        set1= set(list1)
                        if (len(set1)>4):
                            dictWinners[counter] = list1
                            counter = counter+1
counter = 0
for k in dictWinners:
    i=dictWinners[k]
    if L.has_edge(i[0],i[1]) and L.has_edge(i[0],i[2]) and L.has_edge(i[0],i[3]) and L.has_edge(i[0],i[4]) and L.has_edge(i[1],i[2]) and L.has_edge(i[1],i[3]) and L.has_edge(i[1],i[4]) and L.has_edge(i[2],i[3]) and L.has_edge(i[2],i[4]) and L.has_edge(i[3],i[4]):
        counter= counter +1
        countA = 0
        average = 0.0
        for a in range(0,4):
            for b in range(a+1,5):
                average  = average + (L[i[a]][i[b]]["with_win"]+0.0)/ L[i[a]][i[b]]["with_games"]
                countA= countA+1
        average = average /countA
        averages[k] = average
        
import operator
sorted_x = sorted(averages.items(), key=operator.itemgetter(1), reverse= True )
value = 0
s = []
for i,j in sorted_x:
    if value != j:
        s.append(i)
        value = j
counter=1
lastAverage=1
for i in s:
    if lastAverage-averages[i]>0.001:
        print "Team ",counter
        mmrAverage=0
        for k in range(0,5):
            playerk =dictWinners[i][k]
            #https://api.opendota.com/api/players/19757254
            baseurl = "https://api.opendota.com/api/players/"+  str(playerk)
            response = requests.get(baseurl)
            data = response.json()
            soloMMr = data["solo_competitive_rank"]
            if soloMMr==None:
                soloMMr = data["mmr_estimate"]["estimate"]
            
            mmrAverage= mmrAverage+ int(soloMMr)
            print "Id: ",dictWinners[i][k], ", Name: ",prosDict[dictWinners[i][k]], ", Ranking: ", soloMMr
        print "Winrate average: ",averages[i],",Team Average ranking: ", mmrAverage/5.0
        lastAverage= averages[i]
        counter=counter+1


hellllooooo


### The Friendship Paradox: in how many cases does a player has links that have more other links in average?

In [25]:

count =0
for r_Node in H:
    ##r_Node = random.choice(H.nodes())
    averageDegree=0.0
    
    for i in H.neighbors(r_Node):
        averageDegree = averageDegree + H.degree(i);
    averageDegree= averageDegree / len(H.neighbors(r_Node)) 
    
    if (averageDegree > H.degree(r_Node)):
        count = count+1
print "The paradox is true in this case for", count, "times", len(H.nodes())

The paradox is true in this case for 436 times 547


### Exporting everything to the website as a javascript dataSet


#### This is an example of how i exported the rankings of players and names for the website. I did the same for all the data of the website

In [ ]:
f = open('dotaMMR.js', 'w')
f.write("var nodes = [\n")
for node in H.nodes():
    #strings = '{id: ',node,", label: '",prosDict[node],"'", ", title: 'Country: ' + '", country,"'+ '<br>' + 'Team: ' + ' ",teamDict[node],"', value: 22, group:24, x:1, y:2",'},'
    try:
        value = (int(mmrDict[node])-4000)/10
        if value<100:
            value = 100
        if value>450:
            value=800
        mmr = str(int(mmrDict[node]))
        s = "{id: %s, label: '%s', title: 'MMR: ' + '%s' + '<br>' + 'Team: ' + '%s' + '<br>'+'Edges: '+ '%s', value: %s, group: %s, x: -1392.5499, y: 1124.1614},\n" % (str(node), prosDict[node], mmr,  teamDict[node],H.degree(node), value, int(mmr)/1000 )
        print s
        f.write(s)
    except:
        print 'error',node
    
f.write('];')
f.write("var edges = [")
#var edges = [
  #{from: 1, to: 15},
for edge1,edge2 in H.edges():
    s = "{from: %s, to: %s},\n" %(edge1,edge2)
    print s
    f.write(s)    
f.write('];')
f.close()  

## Everything else was done inside the Website with the help of javascript
* showing communities 
* calculating the similarity of players based on their heroes with other players
* showing the mmr distribution and how similar are players with their links based on MMR ranking
* showing second degree neighbors ( neighbors of neighbors)



### Talk about how you've worked with text, including regular expressions, unicode, etc.



#### I downloaded the wordcloud for every pro and created a semtiment score based on the words he wrote in all chat against other players.

#### First loading the datasets of words needed for calculating sentiment

#loading the dataset of words
dataset = open('dataset.txt','r')
happiness_average = {}
for line in dataset:
    
    if line[0]!="#":
        words = line.split()
        happiness_average[words[0]] = words[2]
len(happiness_average)


#### Creating an algorithm for calculating the sentiment score

In [ ]:
import nltk
porter = nltk.PorterStemmer()

def sentiment(words):
    sentimentAverage= 0.0
    counter=0
    for word in words:
        try:
            
            if word in happiness_average:
                sentimentAverage = sentimentAverage + float(happiness_average[word])
                counter +=1##adding word score to sum
            else:
                sentimentAverage += float(happiness_average[porter.stem(word)])
                counter +=1##if word wasn't found, I stem it and try again by adding it
           
                
            #print sentimentAverage
        except:
            l=0
    #print counter
    if counter!=0:
        return sentimentAverage/counter
    else:
        return -1  ###returning -1 as a complaint 


##### Saving the score for each player in a dictionary

In [ ]:
allWords= {}
counter = 0
for i in prosDict:
    try:
        baseurl = "https://api.opendota.com/api/players/%s/wordcloud/" % (str(i))

        response = requests.get(baseurl)
        data = response.json()
    
        s=""
        for k in data['my_word_counts']:
            for j in range(0,data['my_word_counts'][k]):
                s= s + k + " "
        tokens = s.lower().split()
        score = sentiment(tokens)
        allWords[i] = score
        
    except:
        allWords[i] = -200

# 4. Discussion. Think critically about your creation


### What went well?,
More or less, I got I was looking for.I was able to find the players with most edges, the players with highest ranking, the players with most heroes played and also if it matters what ranking you have and you're sentiment is based on that. 

### What is still missing? What could be improved?, Why?

* More analysis for every game played by every player could be done and find if their sentiment changes when they lose or win games
